# Python with MongoDB

Helpful learning resources: https://www.tutorialspoint.com/mongodb/index.htm

## Libraries and Settings

In [ ]:
# Libraries
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

from pymongo import MongoClient

# Settings
import warnings
warnings.filterwarnings("ignore")

# Use dark background for plots
plt.style.use('dark_background')

# Current working directory
print(os.getcwd())

## Connect to the MongoDB server and list databases

In [ ]:
# Connect to the MongoDB server
client = MongoClient("mongodb://mongo:27017/")

# List databases
databases = client.list_database_names()
print("Connected to MongoDB. Databases:", databases)

## Read and insert data

In [ ]:
# Create / Access the specific database and collection
db = client['restaurant_database']
collection = db['restaurant_collection']

# Read data from JSON file
with open('/workspace/restaurant_data.json', 'r') as file:
    example_data = json.load(file)

# Insert data into the collection (similar to tables in an SQL database)
try:
    insert_result = collection.insert_many(example_data)
    print(f"Inserted documents to mongo db")
except Exception as e:
    print(f"Insert error: {e}")


## Define a query using MongoDB Query Language (MQL)

In [ ]:
# Function to query the data
def query_collection(db_name, collection_name, query, client=client):
    if client:
        db = client[db_name]
        collection = db[collection_name]
        results = list(collection.find(query))
        return results
    return []

# Define a query with multiple conditions
query = {
    "properties.addr:city": {"$in": ["Zürich", "Winterthur"]},
    "properties.amenity": "restaurant",
    "properties.cuisine": {"$in": ["burger", "pizza"]}
}

# Execute the query and fetch the results
results = query_collection('restaurant_database', 'restaurant_collection', query)
print("Query result:")
for result in results:
    print(result)

# Convert the results into a Pandas DataFrame
data = []
for result in results:
    properties = result['properties']
    data.append({
        'street': properties.get('addr:street'),
        'housenumber': properties.get('addr:housenumber'),
        'postcode': properties.get('addr:postcode'),
        'city': properties.get('addr:city'),
        'amenity': properties.get('amenity'),
        'cuisine': properties.get('cuisine')
    })

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

## Define a query to aggregate the data using MongoDB Query Language (MQL)

In [ ]:
# Function to aggregate the number of restaurants by cuisine and city
def aggregate_restaurants_by_cuisine_and_city(db_name, collection_name, client=client):
    
    db = client[db_name]
    collection = db[collection_name]
    
    pipeline = [
        {
            "$match": {
                "properties.addr:city": {"$in": ["Zürich", "Winterthur"]},
                "properties.cuisine": {"$in": ["burger", "pizza"]}
            }
        },
        {
            "$group": {
                "_id": {
                    "city": "$properties.addr:city",
                    "cuisine": "$properties.cuisine"
                },
                "count": { "$sum": 1 }
            }
        },
        {
            "$sort": { "count": -1 }
        }
    ]
    
    results = list(collection.aggregate(pipeline))
    return results

# Execute the query and fetch the results
results = aggregate_restaurants_by_cuisine_and_city('restaurant_database', 'restaurant_collection')
print("Query result:")
for result in results:
    print(result)

# Convert the results into a Pandas DataFrame
df = pd.DataFrame(results)

# Normalize the '_id' field to separate columns
df = pd.concat([df.drop(['_id'], axis=1), df['_id'].apply(pd.Series)], axis=1)

# Display the DataFrame
df

## Make bar chart from results

In [ ]:
# Sort values in ascending order
df = df.sort_values(by='count', ascending=True)

# Define colors for each city and cuisine combination
colors = {
    ('Zürich', 'burger'): '#28b463',
    ('Zürich', 'pizza'): '#82e0aa',
    ('Winterthur', 'burger'): '#b7950b',
    ('Winterthur', 'pizza'): '#f7dc6f'
}

# Create a bar chart
fig, ax = plt.subplots()

# Plot data
for city in df['city'].unique():
    city_data = df[df['city'] == city]
    ax.barh(
        city_data['cuisine'] + ' (' + city + ')',
        city_data['count'],
        color=[colors[(city, cuisine)] for cuisine in city_data['cuisine']],
        label=city
    )

# Add labels and title
ax.set_xlabel('Number of Restaurants')
ax.set_ylabel('Cuisine (City)')
ax.set_title('Number of Restaurants by City and Cuisine')

# Create custom legend
handles = [
    plt.Line2D([0], [0], color=list(colors.values())[0], lw=4, label='Zürich (Burger)'),
    plt.Line2D([0], [0], color=list(colors.values())[1], lw=4, label='Zürich (Pizza)'),
    plt.Line2D([0], [0], color=list(colors.values())[2], lw=4, label='Winterthur (Burger)'),
    plt.Line2D([0], [0], color=list(colors.values())[3], lw=4, label='Winterthur (Pizza)')
]
ax.legend(handles=handles, title='City/Cuisine')

# Show the plot
plt.show()

## Drop defined collections and databases

In [ ]:
# List databases
databases = client.list_database_names()
print("Connected to MongoDB. Databases:", databases)

# Drop defined collections and database if exists
db_name = 'restaurant_database'
collection_name = 'restaurant_collection'

db = client[db_name]
if collection_name in db.list_collection_names():
    db.drop_collection(collection_name)
    print(f"Collection '{collection_name}' dropped from database '{db_name}'.")
else:
    print(f"Collection '{collection_name}' does not exist in database '{db_name}'.")

# Drop the database
if db_name in databases:
    client.drop_database(db_name)
    print(f"Database '{db_name}' dropped.")
else:
    print(f"Database '{db_name}' does not exist.")


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')